In [18]:
from socket import *
from config import *
import json
import numpy as np

In [19]:
class SnakeGame(object):
    def __init__(self, seed_num=0):
        super(SnakeGame, self).__init__()
        self.seed_num = seed_num
        print(f"种子号为 {SEED_LIST[seed_num]}")
        self.conn = socket(AF_INET, SOCK_STREAM)
        self.steps = 0
        self.route = ''
        self.directions = ['w', 'a', 's', 'd']
        self.head_abs = (0, 0)
        self.head_view = []
        
    def __del__(self):
        self.close()
        
    def connect(self, host=HOST, port=PORT):
        try:
            self.conn.connect((host, port))
        except ConnectionRefusedError:
            print("请先允许客户端文件 subg.client")
        else:
            print("已经登入了种子岛")

    def pretty_print(self, view):
        h = len(view)
        w = len(view[0])
        
        for i in range(h):
            line = [emoji_table[n] for n in view[i]]
            line_print = "".join(line)
            print(line_print)
                
    def look_around(self, show_view=True):
        self.conn.send(b'g')
        o = self.conn.recv(4096)
        if not o:
            print("游戏结束")
            self.summary()
            if show_view:
                self.pretty_print(self.head_view)
            return -1, -1, []
        else:
            x, y, view = json.loads(o.decode('utf-8')).values()
            print(f"当前蛇头在({x}, {y})")
            if show_view:
                self.pretty_print(view)
            self.head_abs = (x, y)
            self.head_view = np.array(view)

    def move(self, move_char):
        if move_char in self.directions:
            self.steps += 1
            self.route += move_char
            self.conn.send(bytes(move_char, encoding='utf-8'))
            
        else:
            print(f"Invalid move Command! ({move_char} not in 'wasd')")
        
    
    def wander(self, wander_str, show_view=True):
        # check whether wander_str is valid
        for move_char in wander_str:
            if move_char not in self.directions:
                print(f"Invalid move Command! ({move_char} not in 'wasd')")
                break
        
        for move_char in wander_str:
            self.move(move_char)
        
        self.look_around(show_view=show_view)
            
    def findroute_bydir(self, direction='d', distance=1):
        """walk by one direction for 5 distance
        if next step is DEATH, then take the vertical direction
        just use the **first** view
        
        But not take the route
        """    
        head_rel = self.find_head_rel()
        route = ''
            
        print(f"Now take step to Dir {direction} for {distance} distance")
        
        if direction not in self.directions:
            print(f"Invalid direction! ({direction} not in 'wasd')")
        else:
            dir_num = self.directions.index(direction)
            ver_direction = [self.directions[i] for i in range(4) if i != dir_num and i != (dir_num + 2) % 4]
                        
            d = ''
            i = 0
            while i < distance:
                if self.head_view[set_add(head_rel, dir_xy[direction])] not in DEATH:
                    d = direction
                    i += 1
                elif self.head_view[set_add(head_rel, dir_xy[ver_direction[0]])] not in DEATH:
                    d = ver_direction[0]
                elif self.head_view[set_add(head_rel, dir_xy[ver_direction[1]])] not in DEATH:
                    d = ver_direction[1]
                else:
                    print("Dead End WARNING!!!")
                    break
                    
                head_rel = set_add(head_rel, dir_xy[d])
                route += d
                
        return route
            
    def find_head_rel(self):
        head_x_rel, head_y_rel = np.where(self.head_view == HEAD)
        head_x_rel = head_x_rel[0]
        head_y_rel = head_y_rel[0]
        return (head_x_rel, head_y_rel)
    
    def summary(self):
        print(f'The route is {self.route}')
        print(f"All steps is {self.steps}")
        return self.steps, self.route
        
    def close(self):
        self.conn.close()

In [20]:
try:
    if game:
        game.close()
except NameError:
    pass
game = SnakeGame()
game.connect()

种子号为 2020
已经登入了种子岛


In [27]:
game.look_around();

当前蛇头在(5, 1)
🍄⬜⬜⬜🍄🍄
🍄⬜⬜⬜⬜⬜
🍄⬜❤️🍄⬜⬜
🍄🐍⬜🍄⬜⬜
🍄🐉⬜⬜⬜🍄
🍄⬜🍄🍄⬜⬜
🍄⬜❤️🍄🍄⬜
🍄🍄⬜⬜⬜⬜
🍄⬜🍄⬜🍄⬜


In [32]:
filter(lambda x : x == 's', )

Help on class filter in module builtins:

class filter(object)
 |  filter(function or None, iterable) --> filter object
 |  
 |  Return an iterator yielding those items of iterable for which function(item)
 |  is true. If function is None, return the items that are true.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [29]:
route = game.walk_bydir('s', distance=4)
print(route)
if len(route.f) < 4:
    print("Bad Direction")
else:
    game.wander(route)

Now take step to Dir s for 4 distance
ssdsds
Bad Direction


In [17]:
route = game.walk_bydir('s', distance=2)
game.wander(route)

Now take step to Dir s for 2 distance
Dead End WARNING!!!
当前蛇头在(9, 3)
🍄🐍⬜⬜⬜🍄⬜⬜
🍄🐍🍄🍄⬜⬜⬜🍄
🍄🐍🐍🍄🍄⬜🍄⬜
🍄🍄🐍🐍⬜⬜🍄🍄
🍄⬜🍄🐉🍄⬜⬜⬜
🍄🍄❤️🍄⬜⬜⬜⬜
🍄⬜💞⬜⬜⬜⬜❤️
🍄⬜⬜⬜⬜⬜🍄⬜
🍄⬜⬜🍄🍄❤️⬜🍄


In [11]:
game.wander('w', show_view=True)

当前蛇头在(1, 9)
🍄🍄🍄🍄🍄🍄🍄🍄🍄
🍄🍄⬜⬜🐉⬜❤️⬜⬜
⬜⬜⬜🐍🐍⬜⬜⬜⬜
⬜⬜⬜❤️⬜⬜⬜❤️⬜
⬜⬜🍄⬜⬜⬜⬜⬜⬜
🍄⬜⬜⬜🍄⬜⬜❤️⬜


In [12]:
route = game.walk_bydir('d', distance=4)
game.wander(route)

Now take step to Dir d for 4 distance
['w', 's']
当前蛇头在(1, 13)
🍄🍄🍄🍄🍄🍄🍄🍄🍄
⬜🐍🐍🐍🐉⬜❤️🍄⬜
⬜⬜⬜⬜⬜⬜❤️⬜🍄
⬜⬜⬜❤️⬜⬜🍄❤️⬜
⬜⬜⬜⬜⬜⬜⬜🍄❤️
🍄⬜⬜❤️⬜🍄🍄⬜⬜


In [13]:
route = game.walk_bydir('d', distance=4)
game.wander(route)

Now take step to Dir d for 4 distance
['w', 's']
当前蛇头在(3, 17)
🍄🍄🍄🍄🍄🍄🍄🍄🍄
🐍🐍🐍🍄⬜🍄⬜⬜⬜
⬜⬜🐍🐍🍄⬜❤️⬜⬜
⬜⬜🍄🐍🐉⬜⬜⬜💞
⬜⬜⬜🍄❤️⬜❤️🛹🍄
⬜🍄🍄⬜⬜💞🍄💞🍄
💞⬜⬜⬜❤️❤️⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜


In [14]:
route = game.walk_bydir('s', distance=4)
game.wander(route)

Now take step to Dir s for 4 distance
['a', 'd']
当前蛇头在(7, 17)
⬜⬜🍄🐍🐍⬜⬜⬜💞
⬜⬜⬜🍄🐍⬜❤️🛹🍄
⬜🍄🍄⬜🐍💞🍄💞🍄
💞⬜⬜⬜🐍❤️⬜⬜⬜
⬜⬜⬜⬜🐉⬜⬜⬜⬜
⬜💚💞🍄💞💚💞🍄🍄
❤️🍄⬜⬜⬜⬜⬜⬜⬜
🍄⬜❤️⬜⬜💞⬜⬜⬜
💞⬜❤️💞🍄🍄⬜❤️🍄


In [17]:
route = game.walk_bydir('s', distance=4)
game.wander(route)

Now take step to Dir s for 4 distance
['a', 'd']
游戏结束
The route is ddsddddddwddddddsdsdsssssssasaaaassass
All steps is 38
